#### Poetry 환경 구축 및 모듈 설치

```
poetry new "project name" -> Ex) Poetry add sutdy
poetry install
poetry env info 에 나오는 Executable 경로 가져오기기
```
1. Vscode 일 경우

Ctrl + shift + P
select interpreter
+ 가상환경 만들기
Executable의 경로 입력

2. 구글 코랩일 경우
!pip install "패키지 이름" Ex. !pip install numpy pandas 

In [1]:
# 필요 모듈 설치
# !poetry add pandas langchain transformers torch langchain_community
!pip install langchain langchain_community

#### 모듈 호출

In [2]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.embeddings import HuggingFaceEmbeddings
import torch

#### 데이터 가져오기 및 전처리

In [3]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

train.head()
test.head()

,ID,발생일시,사고인지 시간,날씨,기온,습도,공사종류,연면적,층 정보,인적사고,물적사고,공종,사고객체,작업프로세스,장소,부위,사고원인
0,TEST_000,2024-06-03 오전 09:39,정규작업 -,맑음,27℃,53%,건축 / 건축물 / 교정 및 군사시설,"1,990.32㎡","지상 1층, 지하 0층",부딪힘,전도,건축 > 철근콘크리트공사,건설기계 > 콘크리트펌프,타설작업,교정 및 군사시설 / 외부,콘크리트펌프 / 바닥,"펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측..."
1,TEST_001,2024-02-15 오전 09:00,정규작업 -,맑음,5℃,71%,건축 / 건축물 / 운수시설,"349,895㎡","지상 5층, 지하 2층","절단, 베임",없음,건축 > 수장공사,건설공구 > 공구류,절단작업,운수시설 / 내부,공구류 / 핸드그라인더,작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용
2,TEST_002,2024-02-01 오전 09:30,작업 전 -,흐림,5℃,58%,건축 / 건축물 / 공동주택,"171,198㎡","지상 27층, 지하 2층",떨어짐(2미터 미만),없음,건축 > 미장공사,기타 > 기타,이동,공동주택 / 내부,기타 / 바닥,작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐
3,TEST_003,2024-02-06 오후 05:00,정규작업 -,맑음,1℃,87%,건축 / 건축물 / 공동주택,"123,540.98㎡","지상 2층, 지하 35층",넘어짐(물체에 걸림),없음,건축 > 조적공사,건설자재 > 자재,기타,공동주택 / 102동 801호 세대 내부,자재 / 말비계(H:720mm),작업 발판 위 벽돌 잔재를 밟고 넘어짐
4,TEST_004,2024-06-03 오전 11:35,기타 -,흐림,22℃,60%,토목 / 교량 / 도로교량,-,-,떨어짐(2미터 미만),없음,토목 > 교량공사,기타 > 기타,해체작업,도로교량 / 외부,기타 / 바닥,점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생


In [4]:
train['공사종류(대분류)'] = train['공사종류'].str.split('/').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split('/').str[1]
train['공종(대분류)'] = train['공종'].str.split('>').str[0]
train['공종(중분류)'] = train['공종'].str.split('>').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split('>').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split('>').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split('/').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split('/').str[1]
test['공종(대분류)'] = test['공종'].str.split('>').str[0]
test['공종(중분류)'] = test['공종'].str.split('>').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split('>').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split('>').str[1]

In [5]:
# train.apply(
#     lambda row : print(row)
# )
train.columns

Index(['ID', '발생일시', '사고인지 시간', '날씨', '기온', '습도', '공사종류', '연면적', '층 정보',
       '인적사고', '물적사고', '공종', '사고객체', '작업프로세스', '장소', '부위', '사고원인',
       '재발방지대책 및 향후조치계획', '공사종류(대분류)', '공사종류(중분류)', '공종(대분류)', '공종(중분류)',
       '사고객체(대분류)', '사고객체(중분류)'],
      dtype='object')

In [6]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row : {
        "question" : (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중"
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서"
            f"사고객체 '{row['사고객체(대분류)']}', '중분류 : {row['사고객체(중분류)']}'와 관련된 사고가 발생했습니다."
            f"작업프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}' 입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer" : row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

combined_training_data = pd.DataFrame(list(combined_training_data))

combined_test_data = test.apply(
    lambda row : {
        "question" : (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중"
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서"
            f"사고객체 '{row['사고객체(대분류)']}', '중분류 : {row['사고객체(중분류)']}'와 관련된 사고가 발생했습니다."
            f"작업프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}' 입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
    },
    axis=1
)

combined_test_data = pd.DataFrame(list(combined_test_data))

In [7]:
# !poetry add bitsandbytes accelerate>=0.26.0

# !poetry add accelerate==0.26.0 bitsandbytes
# !pip install accelerate==0.26.0 bitsandbytes

### 모델 불러오기

In [8]:
# BitAndBytesConfig 는 양자화를 위한 모듈이다.
bnb_config = BitsAndBytesConfig(
    # 4비트 양자화
    load_in_4bit=True,
    # 이중 양자화 : 속도가 빨라지고 메모리 사용량이 줄어
    bnb_4bit_use_double_quant=True,
    # nf4 방식의 양자화 방식 사용
    bnb_4bit_quant_type="nf4",
    # bfloat16형식으로 계산 수행
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

#### 백터스토어 생성

In [9]:
!pip install FAISS-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.9 MB/s eta 0:00:00


In [10]:
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents  = [
    f"Q: {q1}\nA: {a1}" for q1,a1 in zip(train_questions_prevention, train_answers_prevention)
]
#임베딩 생성
embedding_module_name = "jhgan/ko-sbert-nli"
embedding = HuggingFaceEmbeddings(model_name=embedding_module_name)

vectore_store = FAISS.from_texts(train_documents, embedding)

retriever = vectore_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

<ipython-input-10-cb67b8e077d0>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_module_name)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
create_retrieval_chain?

#### RAG chain 생성

In [52]:
text_generation_pipeline = pipeline(
    model = model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True, # 샘플링 활성화
    max_new_tokens=64,
    temperature=0.1
)

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.
{context}
### 질문:
{question}

[/INST]

"""
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 단순 컨텍스트 결합 방식 사용
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}  # 커스텀 프롬프트 적용
)

Device set to use cuda:0


공사종류 대분류 '건축 ', 중분류 ' 건축물 ' 공사 중공종 대분류 '건축 ', 중분류 ' 철근콘크리트공사' 작업에서사고객체 '건설기계 ', '중분류 :  콘크리트펌프'와 관련된 사고가 발생했습니다.작업프로세스는 '타설작업'이며, 사고 원인은 '펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨' 입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?
공사종류 대분류 '건축 ', 중분류 ' 건축물 ' 공사 중공종 대분류 '건축 ', 중분류 ' 수장공사' 작업에서사고객체 '건설공구 ', '중분류 :  공구류'와 관련된 사고가 발생했습니다.작업프로세스는 '절단작업'이며, 사고 원인은 '작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용' 입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?
공사종류 대분류 '건축 ', 중분류 ' 건축물 ' 공사 중공종 대분류 '건축 ', 중분류 ' 미장공사' 작업에서사고객체 '기타 ', '중분류 :  기타'와 관련된 사고가 발생했습니다.작업프로세스는 '이동'이며, 사고 원인은 '작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐' 입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?
공사종류 대분류 '건축 ', 중분류 ' 건축물 ' 공사 중공종 대분류 '건축 ', 중분류 ' 조적공사' 작업에서사고객체 '건설자재 ', '중분류 :  자재'와 관련된 사고가 발생했습니다.작업프로세스는 '기타'이며, 사고 원인은 '작업 발판 위 벽돌 잔재를 밟고 넘어짐' 입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?
공사종류 대분류 '토목 ', 중분류 ' 교량 ' 공사 중공종 대분류 '토목 ', 중분류 ' 교량공사' 작업에서사고객체 

#### Inference (추론)

In [ ]:
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수 : ", len(combined_test_data))

for idx,row in combined_test_data.iterrows():
  if(idx+1) % 50 == 0 or idx==0:
    print(f"\n샘플 {idx+1}/{len(combined_test_data)} 진행 중...")


    prevention_result = qa_chain.invoke(row['question'])

    result_text = prevention_result['result']
    test_results.append(result_text)

print("\n테스트 실행 완료! 총 결과 수 : ", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수 :  964

샘플 1/964 진행 중...

샘플 50/964 진행 중...

샘플 100/964 진행 중...

샘플 150/964 진행 중...

샘플 200/964 진행 중...


#### Submission (제출)

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-nli"
embedding_model = SentenceTransformer(embedding_model_name, use_auth_token=False)

pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)

submission = pd.read_csv("./sample_submission.csv", encoding = "utf-8")

submission.to_csv('./baseline_submission.csv', index=False, encoding = 'utf-8-sig')